## Using Carrol's (2005) Endogenous Grid Method

This notebook provides code for the endogenous grid method. Although readability over efficiency is prioritized, it still solves the income fluctuations problem (for a given real rate) in less than half a second.

In [14]:
include("rowenhorst_tauchen.jl") # include discretization functions (make sure it is in your working directory)

tauchen (generic function with 1 method)

We begin with a tuple that holds the household parameters.

In [10]:
using Parameters, Plots, LinearAlgebra, Interpolations, BenchmarkTools

Household = @with_kw (apoints = 500, #asset grid
    amax = 200,  # asset max
    beta = 0.98, # discount factor
    alpha = 0.11, # capital share
    deprec = 0.025, # depreciation rate
    gamma = 1, # elasticity of substitution
    bc = 0, # borrowing constraint (must be weakly negative)
    rho = 0.966,  # autocorr of income process
    num_states = 7, # number of states for income process
    sd = 0.13, # stand. dev. of deviation process
    mean = 0, # mean of income process
    uncond_sd = 0.13/sqrt(1-rho^2), # unconditional sd
    transition_matrix = rowenhorst(mean, uncond_sd, rho, num_states)[1], # transition matrix
    ygrid = rowenhorst(mean, uncond_sd, rho, num_states)[2], # grid for income process
    Amat = setgrids(bc, apoints, amax, num_states, ygrid)[1], # asset grid
    Ymat = setgrids(bc, apoints, amax, num_states, ygrid)[2]) # income grid


function setgrids(bc, apoints, amax, num_states, ygrid)
    Amat = [i for i in range(bc, length = apoints, stop= amax), j in 1:num_states]
    Ymat = [j for i = 1:apoints, j in ygrid]
    return Amat, Ymat
end

hh = Household()

Next, some useful functions that will be used in the main EGM function.

In [6]:
# marginal utilities and their inverses

up(c,gamma) = c.^(-gamma)
invup(x,gamma) = x.^(-1/gamma)

# euler equation to get current consumption given future interest rate and future consumption grid

function getc(gamma,beta,transition_matrix; rnext,cnext)
    upnext = beta.*(1+rnext).*up(cnext,gamma)*transition_matrix' # future marginal utility
    c = invup(upnext,gamma) # current consumption
    return c
end

# obtain current assets given consumption today defined on asset grid tomorrow

geta(Amat,Ymat; r,w,c) =  1/(1+r).*(c.+Amat.-w.*Ymat)

# obtain aggregates

function getcapital(; policyfun, dist)
    K = reshape(policyfun, (length(dist),1))
    K = sum(K.*dist)
    return K
end

function getconsumption(; cpolicy,dist)
    C = reshape(cpolicy, (length(dist),1)) 
    C = sum(C.*dist)
    return C
end

getconsumption (generic function with 1 method)

This is the main egm function. It iterates on the Euler equation $c_t = \beta (1+r_{t+1}) \mathbb{E}_t c_{t+1}$ given a guess for the consumption policy function $c_{t+1}$ ("cnext" in the function). Observe that we no longer need a root finding procedure, but still need to interpolate the optimal policy on our defined grids.

In [7]:
# Note: function takes both current and future interest rates as arguments.

function egm(hh; w, cnext, cbinding, r, rnext)
 """
    use endogenous grid method to obtain c_{t} and a_{t} given c_{t+1} 'cnext'
    
    #### Fields
    
    - 'hh': household tuple
    - 'w': wage rate
    - 'cnext': time t+1 consumption grid
    - 'cbinding': consumption grid when borrowing constraint binds
    - 'r': interest rate at time t
    - 'rnext': interest rate at time t+1
    
    #### Returns
    
    - 'c': time t consumption grid
    - 'anext': time t policy function
    
"""
    
@unpack gamma, beta, transition_matrix, Amat, Ymat, num_states = hh

# current policy functions on current grid
c = getc(gamma, beta, transition_matrix; rnext=rnext, cnext=cnext)
a = geta(Amat, Ymat; r=r, w=w, c=c)

cnonbinding = similar(Amat)
    
# get consumption policy function for current grid
for i = 1:num_states
    cnonbinding[:,i] = LinearInterpolation(a[:,i], c[:,i], extrapolation_bc = Line()).(Amat[:,i])
end

# update elements of consumption policy when borrowing constraint binds
# a[1,j] is the level of current assets that induces the borrowing constraint to bind exactly.
# Therefore, whenever current assets are below a[1,j], the borrowing constraint will be STRICTLY binding.
# Note that this uses the monotonicity of the policy rule.

for j = 1:num_states
   c[:,j] = (Amat[:,j] .> a[1,j]) .*cnonbinding[:,j] .+ (Amat[:,j] .<= a[1,j]).*cbinding[:,j]
end
    
# update saving policy function with new consumption function
anext = @. (1+r)*Amat + w*Ymat - c

return c, anext
end

egm (generic function with 1 method)

This is the function that iterates on the EGM function above to solve for the optimal policy rule.

In [8]:
function iterate_egm(hh; r, tol=1e-8, maxiter=1000)
    @unpack deprec, alpha, Amat, Ymat, bc = hh
    
    Z = ((r + deprec)/alpha)^alpha # normalize aggregate income so that Y=1
    w = (1-alpha)*Z*(alpha*Z/(r+deprec))^(alpha/(1-alpha)) # wage rate given guess for r
     
    cnext = @. r*Amat+Ymat*w # initial guess for policy function iteration 
    
    cbinding = @. (1+r)*Amat + w*Ymat + bc # get consumption when borrowing constraint binds
    
    dist = 1
    counter = 0
    
    for i=1:maxiter
        c = egm(hh; w=w, rnext=r, r=r, cnext=cnext, cbinding=cbinding)[1]
        if norm(c-cnext,Inf)<tol 
            println("policy function iteration successful!")
            return egm(hh; w=w, rnext=r, r=r, cnext=c, cbinding=cbinding)
        else
            cnext = c
        end
    end
    
    error("no policy function convergence!")
    
end

iterate_egm (generic function with 1 method)

Solves for the optimal policy in $\approx$ half a second.

In [13]:
@btime iterate_egm(hh, r = 0.1)

policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
policy function iteration successful!
  510.375 ms (73236 allocations: 365.32 MiB)


([0.07096749424518328 0.09474585074712497 … 0.32374756274362093 0.44736616230505283; 0.0802685772211671 0.10402894671986812 … 0.33296332057926986 0.456566049492291; … ; 4.469192997723292 4.494141840127407 … 4.731285018197259 4.858247293339893; 4.478015411449168 4.502964924185298 … 4.740113894500879 4.867079161438543], [0.15810405476459372 0.25061327583238513 … 1.4605614789125752 2.2427427352778837; 0.589684735315664 0.6822119433866962 … 1.8922274846039804 2.6744246116176993; … ; 215.31899678775943 215.41033552292507 … 216.6121422599319 217.390979840716; 215.75105613756062 215.84239420239425 … 217.04419514715536 217.82302973614443])